Summary: 
* I tried some models like Naive Bayes, Decision Tree and the NN from the lab 2. I also tried Random Forest and SVM classifier however they were running like 5 days and the pc crashed.
* For the preprocessing I tried some things I got from the labs and also some tutorials from the web. Tried some combinations of nltk tokenizer, bag of words, max_features and cleaning data.
* I think I failed because my theory is lacking and the way I try preprocessing is not good. Need to practice a lot more to undertsand.
* Actually I enjoyed a lot this activity. Quite a combination of fun and frustration which is interesting.

Some models I tried (I have them spread in different notebooks):
* Naive Bayes - The best accuracy with this model was **0.45091** using countecvectorizer and nltk tokenizer. 
* Decision Tree - The best accuracy with this model was **0.33519** using countvectorizer and nltk tokenizer.
* Random Forest - Never finished -> pc crashed no time to rerun.
* SVM Never finished -> pc crashed no time to rerun. 
* CNN - The best accuracy with this model was **0.4539** using 5000 features 15 epochs, batch size 128 and 5 layers.
* Some of the references I read on the web for some things I tried:
https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34
https://medium.com/voice-tech-podcast/text-classification-using-cnn-9ade8155dfb9
https://towardsdatascience.com/twitter-sentiment-analysis-with-keras-and-nltk-48189967f190
https://towardsdatascience.com/using-lstm-in-twitter-sentiment-analysis-a5d9013b523b
https://huggingface.co/blog/sentiment-analysis-twitter

## 1- Preprocessing the Data

In [1]:
# importing some libraries
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

In [2]:
# importing the data
emotion = pd.read_csv('data/emotion.csv')
identify = pd.read_csv('data/data_identification.csv')
json_object = pd.read_json('data/tweets_DM.json', lines=True)
#json_object = json.loads('data/tweets_DM.json')

In [3]:
# converting to DataFrames
emotion_df = pd.DataFrame(emotion)
identify_df = pd.DataFrame(identify)
json_objectdf = pd.DataFrame(json_object)

In [4]:
# I want to get only the "text" data from the json, this text is inside the source inside the json
source=json_objectdf['_source']
source

0          {'tweet': {'hashtags': ['Snapchat'], 'tweet_id...
1          {'tweet': {'hashtags': ['freepress', 'TrumpLeg...
2          {'tweet': {'hashtags': ['bibleverse'], 'tweet_...
3          {'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...
4          {'tweet': {'hashtags': [], 'tweet_id': '0x2de2...
                                 ...                        
1867530    {'tweet': {'hashtags': ['mixedfeeling', 'butim...
1867531    {'tweet': {'hashtags': [], 'tweet_id': '0x29d0...
1867532    {'tweet': {'hashtags': [], 'tweet_id': '0x2a6a...
1867533    {'tweet': {'hashtags': [], 'tweet_id': '0x24fa...
1867534    {'tweet': {'hashtags': ['Sundayvibes'], 'tweet...
Name: _source, Length: 1867535, dtype: object

In [5]:
# to get a run inside a for loop for all the items in the json
json_objectdf['_source'].count()

1867535

In [6]:
# arrays where the information inside the json will be stored
tweet_array=[] # to store the text data of the tweets
tweet_id=[] # to store the tweet id 

In [7]:
# getting just the text and tweet id from the Json
for i in range(json_objectdf['_source'].count()):
    tweet_array.append(json_objectdf['_source'][i]['tweet']['text'])
    tweet_id.append(json_objectdf['_source'][i]['tweet']['tweet_id'])

In [8]:
# converting to data frames the tweet_array text 
tweet_array_df=pd.DataFrame(tweet_array, columns=['text'])
tweet_array_df.head()

,text
0,"People who post ""add me on #Snapchat"" must be ..."
1,"@brianklaas As we see, Trump is dangerous to #..."
2,"Confident of your obedience, I write to you, k..."
3,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,"""Trust is not the same as faith. A friend is s..."


In [9]:
# converting to data frames the tweet_id
tweet_id_df=pd.DataFrame(tweet_id, columns=['tweet_id'])
tweet_id_df

,tweet_id
0,0x376b20
1,0x2d5350
2,0x28b412
3,0x1cd5b0
4,0x2de201
...,...
1867530,0x316b80
1867531,0x29d0cb
1867532,0x2a6a4f
1867533,0x24faed


In [10]:
# merging the two dataframes, and now we have the data from the json in pandas DF format
merge2=pd.concat([tweet_array_df, tweet_id_df], axis=1)
merge2.head()

,text,tweet_id
0,"People who post ""add me on #Snapchat"" must be ...",0x376b20
1,"@brianklaas As we see, Trump is dangerous to #...",0x2d5350
2,"Confident of your obedience, I write to you, k...",0x28b412
3,Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0
4,"""Trust is not the same as faith. A friend is s...",0x2de201


In [11]:
merge2.shape

(1867535, 2)

In [17]:
# exploration of data inside the CSVs

In [12]:
emotion_df.head(5) # emotion tag of the tweets id  to train the model

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation


In [13]:
identify_df.head(5) # identification tag of the tweets id to train the model

,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train


In [14]:
# merging the CSVs (emotion and identification)
merge1 = pd.merge(emotion_df, identify_df, how='outer')
merge1.shape #same number of rows as the dataframes obtained from the json

(1867535, 3)

In [15]:
merge1.head()

,tweet_id,emotion,identification
0,0x3140b1,sadness,train
1,0x368b73,disgust,train
2,0x296183,anticipation,train
3,0x2bd6e1,joy,train
4,0x2ee1dd,anticipation,train


In [16]:
# now the data is aggregated (tweet id, emotion, identification and text)
merge3=pd.merge(merge1,merge2,how='inner',on='tweet_id')
merge3

,tweet_id,emotion,identification,text
0,0x3140b1,sadness,train,Why Chester? <LH> <LH>
1,0x368b73,disgust,train,@JaredLeto you are the fish that Jonah. Excep...
2,0x296183,anticipation,train,He is coming back again and gonna come again q...
3,0x2bd6e1,joy,train,Dei is really such a beautiful person inside &...
4,0x2ee1dd,anticipation,train,Expressive praise is also an expression of fai...
...,...,...,...,...
1867530,0x2c4dc2,NaN,test,6 year old walks in astounded. Mum! Look how b...
1867531,0x31be7c,NaN,test,Only one week to go until the #inspiringvolunt...
1867532,0x1ca58e,NaN,test,"I just got caught up with the manga for ""My He..."
1867533,0x35c8ba,NaN,test,Speak only when spoken to and make hot ass mus...


In [17]:
# dividing in training and test data for the model
test_data=merge3[merge3['identification']=='test']
training_data=merge3[merge3['identification']=='train']

In [18]:
test_data[0:100]

,tweet_id,emotion,identification,text
1455563,0x28cc61,NaN,test,@Habbo I've seen two separate colours of the e...
1455564,0x2db41f,NaN,test,@FoxNews @KellyannePolls No serious self respe...
1455565,0x2466f6,NaN,test,"Looking for a new car, and it says 1 lady owne..."
1455566,0x23f9e9,NaN,test,@cineworld “only the brave” just out and fount...
1455567,0x1fb4e1,NaN,test,Felt like total dog 💩 going into open gym and ...
...,...,...,...,...
1455658,0x2ee91f,NaN,test,@yelyahwilliams @ParamoreStreet I thought it w...
1455659,0x28bae3,NaN,test,When your gone that's really when they want yo...
1455660,0x25a49d,NaN,test,The @Patriots D making @nyjets look like Super...
1455661,0x368d58,NaN,test,@Macys What the hell? Standing in men's furnis...


In [19]:
training_data.head(100)

,tweet_id,emotion,identification,text
0,0x3140b1,sadness,train,Why Chester? <LH> <LH>
1,0x368b73,disgust,train,@JaredLeto you are the fish that Jonah. Excep...
2,0x296183,anticipation,train,He is coming back again and gonna come again q...
3,0x2bd6e1,joy,train,Dei is really such a beautiful person inside &...
4,0x2ee1dd,anticipation,train,Expressive praise is also an expression of fai...
...,...,...,...,...
95,0x356204,joy,train,Appreciate everyone who purchased a cd today! ...
96,0x32ef9f,joy,train,just realized holes is on netflix so happy fuc...
97,0x33228f,disgust,train,@LowkoTV This is..... beyond words <LH>
98,0x206046,anticipation,train,<LH> is about the intervention of <LH> into ou...


In [20]:
test_data[test_data['tweet_id']=='0x26289a'] # to verify if the merging was ok

,tweet_id,emotion,identification,text
1793332,0x26289a,NaN,test,"In these tough times, who do YOU turn to as yo..."


In [21]:
# saving training and testing data as CSV files to use later
training_data.to_csv('data/training_data.csv', index=False, header=True)
test_data.to_csv('data/test_data.csv', index=False, header=True)

## 2- Modeling the Data

1) Naive Bayes

In [31]:
# Naive Bayes
# building the model
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
t=time()

# Transforming train data with count vectorizer and tfidf vectorizer, I tried with both
#  also changing tokenizer=nltk.word_tokenize and max_features combinations
#cv = CountVectorizer()
tfid = TfidfVectorizer()
X_train = tfid.fit_transform(training_data['text'])
y_train = training_data['emotion']

X_test = tfid.transform(test_data['text'])

# data dimension
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

X_train.shape:  (1455563, 794247)
y_train.shape:  (1455563,)
X_test.shape:  (411972, 794247)
test time:  51.724s


In [32]:
t=time()

# building naive bayes model with Multinomial distribution 
nbm = MultinomialNB()

# training the model
nbm = nbm.fit(X_train, y_train)

# doing some predictions
y_train_pred = nbm.predict(X_train)
y_test_pred = nbm.predict(X_test)

test_time = time() - t

print("test time:  %0.3fs" % test_time)
print(y_test_pred)

test time:  8.127s
['joy' 'sadness' 'joy' ... 'joy' 'joy' 'joy']


In [33]:
# converting predictions to dataframes
y_test_pred_df=pd.DataFrame(y_test_pred)
y_test_pred_df.shape

(411972, 1)

In [34]:
y_test_pred_df.value_counts()

joy             365850
sadness          24826
anticipation     15635
disgust           3491
trust             1735
fear               430
surprise             3
anger                2
dtype: int64

In [25]:
test_data.shape

(411972, 4)

In [35]:
# saving to CSVs to later submit in the Kaggle the tweet id and the emotion predicted for that id
y_test_pred_df.to_csv('data/test_pred_tfid_notokenizer.csv', index=False, header=True)
test_data['tweet_id'].to_csv('data/tweet_id.csv', index=False, header=True)

2) Decision Tree

In [50]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
cv2 = CountVectorizer(tokenizer=nltk.word_tokenize)
cv2.fit(training_data['text'])

# training & testing data
X_train2 = cv2.transform(training_data['text'])
y_train2 = training_data['emotion']

X_test2 = cv2.transform(test_data['text'])

# data dimension 
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)

C:\Users\Ellis\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


X_train.shape:  (1455563, 794247)
y_train.shape:  (1455563,)
X_test.shape:  (411972, 794247)


In [51]:
# build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=1)

# training
DT_model = DT_model.fit(X_train2, y_train2)

# predicting
y_train_pred2 = DT_model.predict(X_train2)
y_test_pred2 = DT_model.predict(X_test2)

# pred result
y_test_pred2[:10]

array(['disgust', 'joy', 'sadness', 'joy', 'joy', 'joy', 'joy', 'anger',
       'anticipation', 'anticipation'], dtype=object)

In [ ]:
# saving to CSVs to later submit in the Kaggle

In [52]:
y_test_pred2_df=pd.DataFrame(y_test_pred2)
y_test_pred2_df.shape

(411972, 1)

In [54]:
y_test_pred2_df.to_csv('data/test_pred2.csv', index=False, header=True)
test_data['tweet_id'].to_csv('data/tweet_id.csv', index=False, header=True)

In [45]:
y_test_pred3_df=pd.DataFrame(y_test_pred3)
y_test_pred3_df.shape

(411972, 1)

3) Random Forest

In [37]:
# Random Forest
# This one could not finish, 
# I think to many nodes because I tried with the whole dataset hence too many features
# Try with lest features

t=time()
# building the model

from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

# Transforming train data with count vectorizer 
cv = CountVectorizer(tokenizer=nltk.word_tokenize)
X_train4 = cv.fit_transform(training_data['text'])
y_train4 = training_data['emotion']

X_test4 = cv.transform(test_data['text'])



# take a look at data dimension
print('X_train.shape: ', X_train4.shape)
print('y_train.shape: ', y_train4.shape)
print('X_test.shape: ', X_test4.shape)


test_time = time() - t
print("test time:  %0.3fs" % test_time)

X_train.shape:  (1455563, 994729)
y_train.shape:  (1455563,)
X_test.shape:  (411972, 994729)
test time:  527.589s


In [ ]:
# Random Forest
#https://towardsdatascience.com/a-practical-guide-to-implementing-a-random-forest-classifier-in-python-979988d8a263
from time import time
from sklearn.ensemble import RandomForestClassifier
t=time()

# building randomforest
rfc = RandomForestClassifier()

# training the model
rfc = rfc.fit(X_train4, y_train4)

# doing some predictions
y_train_pred4 = rfc.predict(X_train4)
y_test_pred4 = rfc.predict(X_test4)

print (rfc.score(X_train4, y_train4))
print(y_test_pred4[:10])
test_time = time() - t
print("test time:  %0.3fs" % test_time)